In [32]:
import pandas as pd
NBA = pd.read_csv('wildcard_data.csv') #data for last 21 days
NBA.rename(columns={'form':'Form','Position':'POS'}, inplace=True)
NBA['Price'] = NBA['Price'].astype(float)
NBA['Form'] = NBA['Form'].astype(float)
NBA=NBA.loc[NBA.Form>0]

In [33]:
#cleaning data
PG=NBA.loc[NBA.POS=='PG']
SG=NBA.loc[NBA.POS=='SG']
SF=NBA.loc[NBA.POS=='SF']
PF=NBA.loc[NBA.POS=='PF']
C=NBA.loc[NBA.POS=='C']
PG=PG.sort_values(by=['Price'], ascending=False).reset_index(drop=True)
SG=SG.sort_values(by=['Price'], ascending=False).reset_index(drop=True)
SF=SF.sort_values(by=['Price'], ascending=False).reset_index(drop=True)
PF=PF.sort_values(by=['Price'], ascending=False).reset_index(drop=True)
C=C.sort_values(by=['Price'], ascending=False).reset_index(drop=True)

In [34]:
PG_value=[]
SG_value=[]
SF_value=[]
PF_value=[]
C_value=[]
list_positions=['PG','SG','SF','PF','C']
def calculate_value(pos):
    if pos=='PG':
        pos_list=PG
    if pos=='SG':
        pos_list=SG
    if pos=='SF':
        pos_list=SF
    if pos=='PF':
        pos_list=PF
    if pos=='C':
        pos_list=C
    for i in range(len(pos_list.index)-1):
        change_in_Form=0
        change_in_price=1
        next_highest_scoring_player=pos_list.loc[pos_list.Form==max(pos_list.Form[(i+1):])]
        next_highest_scoring_player=next_highest_scoring_player.reset_index()
        price=(next_highest_scoring_player.Price)
        try:
            if len(price)==2:
                price=pd.Series(price[1])
        except Exception:
            pass
        if price.size==0:
            price=[0]
        change_in_Form=float(pos_list['Form'][i])-float(max(pos_list.Form[(i+1):]))
        change_in_price=float(pos_list['Price'][i])-price[0]
        if change_in_price==0:
            change_in_price=1
        value=round((change_in_Form/change_in_price),2)
        if pos=='PG':
            PG_value.append(value)

        if pos=='SG':
            SG_value.append(value)

        if pos=='SF':
            SF_value.append(value)


        if pos=='PF':
            PF_value.append(value)

        if pos=='C':
            C_value.append(value)

for pos in list_positions:
    calculate_value(pos)
#setting the last player in each position's 'value' to 0, they would never be in the team anyway
PG_value.append(0)
SG_value.append(0)
SF_value.append(0)
PF_value.append(0)
C_value.append(0)
#adding values to df's and converting them to float
PG['Value']=PG_value
SG['Value']=SG_value
SF['Value']=SF_value
PF['Value']=PF_value
C['Value']=C_value
PG['Value'] = PG['Value'].astype(float)
SG['Value'] = SG['Value'].astype(float)
SF['Value'] = SF['Value'].astype(float)
PF['Value'] = PF['Value'].astype(float)
C['Value'] = C['Value'].astype(float)


In [35]:
#removing players with -ve value, i.e. players with replacements who are cheaper and score more points (change in form -ve/ change in price +ve)
PG=PG.loc[PG.Value>0]
PG=PG.reset_index(drop=True)
SG=SG.loc[SG.Value>0]
SG=SG.reset_index(drop=True)
SF=SF.loc[SF.Value>0]
SF=SF.reset_index(drop=True)
PF=PF.loc[PF.Value>0]
PF=PF.reset_index(drop=True)
C=C.loc[C.Value>0]
C=C.reset_index(drop=True)

In [36]:
#creating functions to be used later, calcuate team price, pts and dropping and replacing players
def get_team_value(team):
    values=team.Price
    team_value=values.sum()
    return team_value
def get_team_pts(team):
    pts=team.Form
    team_pts=pts.sum()
    return team_pts
def drop_and_replace(df1,df2,p1_idx,p2_idx):
    df1=df1.reset_index(drop=True)
    df1=df1.sort_values(by=['Value'], ascending=False)
    df1=df1.drop([p1_idx])
    new_player=df2.iloc[p2_idx]
    new_player=new_player.to_frame().transpose()
    df1=pd.concat([df1, new_player])
    return df1

In [47]:
PG=PG.sort_values(by=['Form'], ascending=False)
SG=SG.sort_values(by=['Form'], ascending=False)
SF=SF.sort_values(by=['Form'], ascending=False)
PF=PF.sort_values(by=['Form'], ascending=False)
C=C.sort_values(by=['Form'], ascending=False)
best_team=pd.concat([PG.iloc[:2],SG.iloc[:2],SF.iloc[:2],PF.iloc[:2],C.iloc[:1]], ignore_index='True')
pg_idx=2
sg_idx=2
sf_idx=2
pf_idx=2
c_idx=2
while get_team_value(best_team)>133.8:
    best_team=best_team.sort_values(by=['Value'], ascending=False)
    best_team=best_team.reset_index(drop=True)
    worst_value_player=best_team.iloc[8]
    try:
        if worst_value_player.POS=='PG':
            best_team=drop_and_replace(best_team,PG,8,pg_idx)
            pg_idx+=1
        if worst_value_player.POS=='SG':
            best_team=drop_and_replace(best_team,SG,8,sg_idx)
            sg_idx+=1
        if worst_value_player.POS=='SF':
            best_team=drop_and_replace(best_team,SF,8,sf_idx)
            sf_idx+=1
        if worst_value_player.POS=='PF':
            best_team=drop_and_replace(best_team,PF,8,pf_idx)
            pf_idx+=1
        if worst_value_player.POS=='C':
            best_team=drop_and_replace(best_team,C,8,c_idx)
            c_idx+=1
    except Exception:
        pass
best_team=best_team.sort_values(by=['Form'], ascending=False)

In [46]:
# this was also done using data from the last 30 days, the result is below
#30 day data team below
print(f"Team pts: {round(get_team_pts(best_team),2)}, Team Price: {get_team_value(best_team)}")
best_team

Team pts: 444.2, Team Price: 133.1


,Player,POS,Price,Form,Value
7,Luka Doncic,PG,21.4,68.175,4.65
5,Giannis Antetokounmpo,PF,20.3,64.275,5.54
4,Trae Young,PG,17.0,54.7,6.36
3,Scottie Barnes,SF,14.9,47.925,7.27
2,Kyrie Irving,SG,15.5,46.2,8.17
6,Bam Adebayo,C,14.3,45.125,4.9
0,Jalen Johnson,PF,8.3,40.5,41.92
1,Coby White,SG,9.2,39.325,12.39
4,Franz Wagner,SF,12.2,37.975,2.31


In [48]:
#best team in last 21 days
print(f"Team pts: {round(get_team_pts(best_team_short),2)}, Team Price: {get_team_value(best_team_short)}")
best_team_short

Team pts: 435.52, Team Price: 131.6


,Player,POS,Price,Form,Value
6,Joel Embiid,C,21.0,66.850,4.80
7,Giannis Antetokounmpo,PF,20.3,63.400,4.79
5,Shai Gilgeous-Alexander,PG,18.8,56.275,5.07
2,Kyrie Irving,SG,15.5,51.050,18.29
4,Paolo Banchero,PF,14.1,49.475,8.41
8,Cade Cunningham,PG,14.5,46.750,1.52
1,Miles Bridges,SF,11.8,40.100,24.25
3,Caris LeVert,SF,9.0,33.375,9.00
0,Brandin Podziemski,SG,6.6,28.250,58.75


I ended up making 3 changes to 30 day team since I generally thought  it better represented player performance than the 21 day version, brought in Deni Avdija for Barnes due to his change in form in recent weeks, as well as saving funds which were used later, Shaedon Sharpe for Franz Wagner since Wagner was injured and this freed funds up to get Joel Embiid for Bam Adebayo